In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import json
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


%matplotlib inline

In [2]:
path = '/Volumes/GoogleDrive/My Drive/wish_local/DSF-1436 Fusion Incrementality Analysis/'

In [20]:
ios = pd.read_csv(path+'data/dsf1436_ios_dau_monthly_gmv.csv')#, parse_dates=['month'])
ios = ios.rename(columns={'ios_wish_blue_fusion_bucket': 'bucket'})
ios = ios.sort_values(['month', 'bucket'])
ios['bucket'] = ios['bucket'].apply(lambda x: 'IOS_'+ x.upper())
ios.head(1)

,month,bucket,dusers0,dusers,order_users,order_nusers,pdp_users,avg_orders,std_orders,avg_forders,avg_gmv,std_gmv,avg_pdp,std_pdp,avg_imp,std_imp,time
4,2021-04,IOS_CONTROL,3565406,3565405,170472,8926,2326268,0.126766,0.859251,0.002955,1.695718,13.300395,5.285319,12.993731,196.820919,558.941721,1628201397


In [14]:
android = pd.read_csv(path+'data/dsf1436_android_dau_monthly_gmv.csv')#, parse_dates=['month'])
android = android.rename(columns={'wish_blue_fusion_bucket': 'bucket'})
android = android.sort_values(['month', 'bucket'])
android['bucket'] = android['bucket'].apply(lambda x: 'ANDROID_'+ x.upper())

In [15]:
ios.shape, android.shape

((8, 17), (8, 17))

In [18]:
df = ios.append(android).sort_values(['month', 'bucket'])
df.shape

(16, 17)

In [19]:
df

,month,bucket,dusers0,dusers,order_users,order_nusers,pdp_users,avg_orders,std_orders,avg_forders,avg_gmv,std_gmv,avg_pdp,std_pdp,avg_imp,std_imp,time
5,2021-04,ANDROID_CONTROL,6328014,6328013,360900,15763,4630497,0.151861,0.965265,0.001882,1.971166,14.541391,7.006181,16.092921,264.438004,670.679653,1628225022
4,2021-04,ANDROID_SHOW,120644432,120644425,6891614,298333,88234674,0.152668,0.961982,0.018283,1.949224,14.329678,6.979089,15.989134,263.182445,874.990855,1628225022
4,2021-04,IOS_CONTROL,3565406,3565405,170472,8926,2326268,0.126766,0.859251,0.002955,1.695718,13.300395,5.285319,12.993731,196.820919,558.941721,1628201397
0,2021-04,IOS_SHOW,67405121,67405117,3222683,168040,44069015,0.127982,0.892600,0.014013,1.682146,13.829706,5.271696,13.374529,196.133093,547.557997,1628201397
1,2021-05,ANDROID_CONTROL,6314464,6314464,336310,13634,4604049,0.138007,0.885198,0.002203,1.808508,14.148521,6.629633,15.356282,251.196263,634.165335,1628225022
3,2021-05,ANDROID_SHOW,120668165,120668163,6438000,252924,87878066,0.139229,0.899450,0.020284,1.796113,13.674724,6.592494,15.312115,249.931959,766.070739,1628225022
5,2021-05,IOS_CONTROL,3521527,3521527,160959,7725,2220044,0.120842,0.846369,0.003442,1.656361,13.690906,4.978854,12.252784,185.987011,496.373166,1628201397
3,2021-05,IOS_SHOW,66655315,66655314,3037059,147002,42135937,0.121703,0.869456,0.016004,1.638625,14.150451,4.978942,13.298653,186.538293,538.982429,1628201397
7,2021-06,ANDROID_CONTROL,5366595,5366595,271181,11262,3876382,0.149538,1.004591,0.002538,1.925815,15.229427,6.427937,15.600350,312.218299,801.159563,1628225022
0,2021-06,ANDROID_SHOW,102923161,102923157,5208154,208214,74270883,0.151884,1.026778,0.024195,1.917672,15.632095,6.402566,15.195461,312.416176,1169.785057,1628225022


# proportion of dau who place order(s) and who click pdp

In [29]:
prop = df[['month', 'bucket', 'dusers', 'order_users', 'pdp_users']].copy(deep=True)
prop['order_users_pct'] = prop['order_users'] / prop['dusers']
prop['pdp_users_pct'] = prop['pdp_users'] / prop['dusers']

## calculate diff

In [25]:
def show_control_diff(x, data, c, diff_pct=False):
    """
    x: row
    data: original dataframe
    c: column to calculate diff
    diff_pct: False is return absoluate diff, True is to return diff %
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        control_value = data[(data['month']==x['month']) & \
                             (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][c].iloc[0]
        show_value = x[c]
        if diff_pct:
            return (show_value - control_value) / control_value
        else:
            return show_value - control_value

In [31]:
prop

,month,bucket,dusers,order_users,pdp_users,order_users_pct,pdp_users_pct
5,2021-04,ANDROID_CONTROL,6328013,360900,4630497,0.057032,0.731746
4,2021-04,ANDROID_SHOW,120644425,6891614,88234674,0.057123,0.731361
4,2021-04,IOS_CONTROL,3565405,170472,2326268,0.047813,0.652455
0,2021-04,IOS_SHOW,67405117,3222683,44069015,0.047811,0.653793
1,2021-05,ANDROID_CONTROL,6314464,336310,4604049,0.053260,0.729127
3,2021-05,ANDROID_SHOW,120668163,6438000,87878066,0.053353,0.728262
5,2021-05,IOS_CONTROL,3521527,160959,2220044,0.045707,0.630421
3,2021-05,IOS_SHOW,66655314,3037059,42135937,0.045564,0.632147
7,2021-06,ANDROID_CONTROL,5366595,271181,3876382,0.050531,0.722317
0,2021-06,ANDROID_SHOW,102923157,5208154,74270883,0.050602,0.721615


In [45]:
for c in ['order_users_pct', 'pdp_users_pct']:
    prop[c+'_diff'] = prop.apply(lambda x: \
                show_control_diff(x, prop, c, diff_pct=False), axis=1)
temp = list(zip(['order_users_pct', 'pdp_users_pct'], \
                [c+'_diff' for c in ['order_users_pct', 'pdp_users_pct']]))
temp_cols = [l1 for l2 in temp for l1 in l2]
prop[['month', 'bucket', 'dusers'] + temp_cols]

,month,bucket,dusers,order_users_pct,order_users_pct_diff,pdp_users_pct,pdp_users_pct_diff
5,2021-04,ANDROID_CONTROL,6328013,0.057032,NaN,0.731746,NaN
4,2021-04,ANDROID_SHOW,120644425,0.057123,0.000091,0.731361,-0.000384
4,2021-04,IOS_CONTROL,3565405,0.047813,NaN,0.652455,NaN
0,2021-04,IOS_SHOW,67405117,0.047811,-0.000002,0.653793,0.001338
1,2021-05,ANDROID_CONTROL,6314464,0.053260,NaN,0.729127,NaN
3,2021-05,ANDROID_SHOW,120668163,0.053353,0.000093,0.728262,-0.000865
5,2021-05,IOS_CONTROL,3521527,0.045707,NaN,0.630421,NaN
3,2021-05,IOS_SHOW,66655314,0.045564,-0.000144,0.632147,0.001726
7,2021-06,ANDROID_CONTROL,5366595,0.050531,NaN,0.722317,NaN
0,2021-06,ANDROID_SHOW,102923157,0.050602,0.000071,0.721615,-0.000702


## proportion test

In [46]:
from statsmodels.stats.proportion import proportions_ztest
# https://www.statsmodels.org/devel/generated/statsmodels.stats.proportion.proportions_ztest.html

def show_control_prop_test(x, data, pct_col, nobs_col, is_count=False, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    pct_col: column name of numerator or the percentage calcualted
    nobs_col: column name of denominator
    is_count: False if pct_col is percentage, True if pct_col is the integer (numerator)
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        n_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][nobs_col].iloc[0]
        n_show = x[nobs_col]
        if is_count: 
            count_control = data[(data['month']==x['month']) & \
                                 (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][pct_col].iloc[0]
            count_show = x[pct_col]
        else:
            count_control = int(round(data[(data['month']==x['month']) & \
                                     (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][pct_col].iloc[0] * n_control,0))
            count_show = int(round(x[pct_col] * n_show, 0))
        
        stat, pval = proportions_ztest((count_control,count_show), (n_control,n_show), alternative=alternative)
        
        return round(pval, 2)

In [47]:
for c in ['order_users_pct', 'pdp_users_pct']:
    prop[c+'_pval'] = prop.apply(lambda x: \
         show_control_prop_test(x, prop, c, 'dusers', is_count=False, alternative='two-sided')
         , axis=1)
    prop[c+'_diff'] = prop.apply(lambda x: f"{round(x[c+'_diff']*100, 2)}" + \
                             ("" if np.isnan(x[c+'_diff']) else "%") + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip(['order_users_pct', 'pdp_users_pct'], [c+'_diff' for c in ['order_users_pct', 'pdp_users_pct']])#, [c+'_test' for c in prop_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

prop[['month', 'bucket', 'dusers']+temp_cols]#.to_clipboard()

,month,bucket,dusers,order_users_pct,order_users_pct_diff,pdp_users_pct,pdp_users_pct_diff
5,2021-04,ANDROID_CONTROL,6328013,0.057032,nan,0.731746,nan
4,2021-04,ANDROID_SHOW,120644425,0.057123,0.01%,0.731361,-0.04%***
4,2021-04,IOS_CONTROL,3565405,0.047813,nan,0.652455,nan
0,2021-04,IOS_SHOW,67405117,0.047811,-0.0%,0.653793,0.13%***
1,2021-05,ANDROID_CONTROL,6314464,0.053260,nan,0.729127,nan
3,2021-05,ANDROID_SHOW,120668163,0.053353,0.01%,0.728262,-0.09%***
5,2021-05,IOS_CONTROL,3521527,0.045707,nan,0.630421,nan
3,2021-05,IOS_SHOW,66655314,0.045564,-0.01%,0.632147,0.17%***
7,2021-06,ANDROID_CONTROL,5366595,0.050531,nan,0.722317,nan
0,2021-06,ANDROID_SHOW,102923157,0.050602,0.01%,0.721615,-0.07%***


In [50]:
def color_pval(x):
    if '*' in str(x):
        if '-' in str(x):
            return "background-color: pink"
        else:
            return "background-color: lightgreen"
    return "background-color: white"

prop[['month', 'bucket', 'dusers']+temp_cols].reset_index().style.applymap(color_pval)

,index,month,bucket,dusers,order_users_pct,order_users_pct_diff,pdp_users_pct,pdp_users_pct_diff
0,5,2021-04,ANDROID_CONTROL,6328013,0.057032,nan,0.731746,nan
1,4,2021-04,ANDROID_SHOW,120644425,0.057123,0.01%,0.731361,-0.04%***
2,4,2021-04,IOS_CONTROL,3565405,0.047813,nan,0.652455,nan
3,0,2021-04,IOS_SHOW,67405117,0.047811,-0.0%,0.653793,0.13%***
4,1,2021-05,ANDROID_CONTROL,6314464,0.053260,nan,0.729127,nan
5,3,2021-05,ANDROID_SHOW,120668163,0.053353,0.01%,0.728262,-0.09%***
6,5,2021-05,IOS_CONTROL,3521527,0.045707,nan,0.630421,nan
7,3,2021-05,IOS_SHOW,66655314,0.045564,-0.01%,0.632147,0.17%***
8,7,2021-06,ANDROID_CONTROL,5366595,0.050531,nan,0.722317,nan
9,0,2021-06,ANDROID_SHOW,102923157,0.050602,0.01%,0.721615,-0.07%***


# Avg orders, gmv, pdp and imp per dau

In [52]:
og_cols = ['avg_orders',
             'std_orders',
             'avg_gmv',
             'std_gmv',
             'avg_pdp',
             'std_pdp',
             'avg_imp',
             'std_imp']

In [53]:
og = df[['month', 'bucket', 'dusers']+og_cols].copy(deep=True)
og

,month,bucket,dusers,avg_orders,std_orders,avg_gmv,std_gmv,avg_pdp,std_pdp,avg_imp,std_imp
5,2021-04,ANDROID_CONTROL,6328013,0.151861,0.965265,1.971166,14.541391,7.006181,16.092921,264.438004,670.679653
4,2021-04,ANDROID_SHOW,120644425,0.152668,0.961982,1.949224,14.329678,6.979089,15.989134,263.182445,874.990855
4,2021-04,IOS_CONTROL,3565405,0.126766,0.859251,1.695718,13.300395,5.285319,12.993731,196.820919,558.941721
0,2021-04,IOS_SHOW,67405117,0.127982,0.892600,1.682146,13.829706,5.271696,13.374529,196.133093,547.557997
1,2021-05,ANDROID_CONTROL,6314464,0.138007,0.885198,1.808508,14.148521,6.629633,15.356282,251.196263,634.165335
3,2021-05,ANDROID_SHOW,120668163,0.139229,0.899450,1.796113,13.674724,6.592494,15.312115,249.931959,766.070739
5,2021-05,IOS_CONTROL,3521527,0.120842,0.846369,1.656361,13.690906,4.978854,12.252784,185.987011,496.373166
3,2021-05,IOS_SHOW,66655314,0.121703,0.869456,1.638625,14.150451,4.978942,13.298653,186.538293,538.982429
7,2021-06,ANDROID_CONTROL,5366595,0.149538,1.004591,1.925815,15.229427,6.427937,15.600350,312.218299,801.159563
0,2021-06,ANDROID_SHOW,102923157,0.151884,1.026778,1.917672,15.632095,6.402566,15.195461,312.416176,1169.785057


## calculate diff

In [66]:
for c in og_cols:
    og[c+'_diff'] = og.apply(lambda x: show_control_diff(x, og, c, diff_pct=True), axis=1)
    
temp = zip(['avg_orders', 'avg_gmv', 'avg_pdp', 'avg_imp'],\
           [c+'_diff' for c in ['avg_orders', 'avg_gmv', 'avg_pdp', 'avg_imp']])
temp_cols = [l1 for l2 in temp for l1 in l2]

og[['month', 'bucket', 'dusers']+temp_cols].head()

,month,bucket,dusers,avg_orders,avg_orders_diff,avg_gmv,avg_gmv_diff,avg_pdp,avg_pdp_diff,avg_imp,avg_imp_diff
5,2021-04,ANDROID_CONTROL,6328013,0.151861,NaN,1.971166,NaN,7.006181,NaN,264.438004,NaN
4,2021-04,ANDROID_SHOW,120644425,0.152668,0.005313,1.949224,-0.011131,6.979089,-0.003867,263.182445,-0.004748
4,2021-04,IOS_CONTROL,3565405,0.126766,NaN,1.695718,NaN,5.285319,NaN,196.820919,NaN
0,2021-04,IOS_SHOW,67405117,0.127982,0.009587,1.682146,-0.008003,5.271696,-0.002578,196.133093,-0.003495
1,2021-05,ANDROID_CONTROL,6314464,0.138007,NaN,1.808508,NaN,6.629633,NaN,251.196263,NaN


## t test

In [67]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# https://github.com/scipy/scipy/blob/v1.7.1/scipy/stats/stats.py#L5879-

from scipy.stats import distributions

def _unequal_var_ttest_denom(v1, n1, v2, n2):
    vn1 = v1 / n1
    vn2 = v2 / n2
    with np.errstate(divide='ignore', invalid='ignore'):
        dof = (vn1 + vn2)**2 / (vn1**2 / (n1 - 1) + vn2**2 / (n2 - 1))

    # If dof is undefined, variances are zero (assumes n1 > 0 & n2 > 0).
    # Hence it doesn't matter what dof is as long as it's not NaN.
    dof = np.where(np.isnan(dof), 1, dof)
    denom = np.sqrt(vn1 + vn2)
    return dof, denom

def _ttest_finish(dof, t, alternative):
    """Common code between all 3 t-test functions."""
    if alternative == 'less':
        prob = distributions.t.cdf(t, dof)
    elif alternative == 'greater':
        prob = distributions.t.sf(t, dof)
    elif alternative == 'two-sided':
        prob = 2 * distributions.t.sf(np.abs(t), dof)
    else:
        raise ValueError("alternative must be "
                         "'less', 'greater' or 'two-sided'")
    return t, prob

def _ttest_ind_from_stats(mean1, mean2, denom, dof, alternative):

    d = mean1 - mean2
    with np.errstate(divide='ignore', invalid='ignore'):
        t = np.divide(d, denom)
    t, prob = _ttest_finish(dof, t, alternative)

    return (t, prob)



def unequal_var_ttest(v1, v2, n1, n2, m1, m2, alternative='two-sided'):
    dof, denom = _unequal_var_ttest_denom(v1, n1, v2, n2)
    t, pval = _ttest_ind_from_stats(m1, m2, denom, dof, alternative)
    
    return t, pval

In [68]:
def show_control_t_test(x, data, avg_col, std_col, n_col, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        v_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][std_col].iloc[0] ** 2
        v_show = x[std_col] ** 2
        
        n_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][n_col].iloc[0]
        n_show = x[n_col]
        
        m_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][avg_col].iloc[0]
        m_show = x[avg_col]
        
        t, pval = unequal_var_ttest(v_control, v_show, n_control, n_show, m_control, m_show, alternative)
        
        return round(pval, 4)


In [71]:
for c in ['orders', 'gmv', 'pdp', 'imp']:
    og[c+'_pval'] = og.apply(lambda x: \
                                 show_control_t_test(x, data=og, \
                                                     avg_col='avg_'+c, \
                                                     std_col='std_'+c, \
                                                     n_col='dusers', \
                                                     alternative='two-sided') , axis=1)

    og['avg_'+c+'_diff2'] = og.apply(lambda x: f"{round(x['avg_'+c+'_diff']*100, 2)}%" + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip(['avg_orders', 'avg_gmv', 'avg_pdp', 'avg_imp'], \
           [c+'_diff2' for c in ['avg_orders', 'avg_gmv', 'avg_pdp', 'avg_imp']])#, [c+'_test' for c in og_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

og[['month', 'bucket', 'dusers']+temp_cols]

,month,bucket,dusers,avg_orders,avg_orders_diff2,avg_gmv,avg_gmv_diff2,avg_pdp,avg_pdp_diff2,avg_imp,avg_imp_diff2
5,2021-04,ANDROID_CONTROL,6328013,0.151861,nan%,1.971166,nan%,7.006181,nan%,264.438004,nan%
4,2021-04,ANDROID_SHOW,120644425,0.152668,0.53%***,1.949224,-1.11%***,6.979089,-0.39%***,263.182445,-0.47%***
4,2021-04,IOS_CONTROL,3565405,0.126766,nan%,1.695718,nan%,5.285319,nan%,196.820919,nan%
0,2021-04,IOS_SHOW,67405117,0.127982,0.96%***,1.682146,-0.8%,5.271696,-0.26%,196.133093,-0.35%***
1,2021-05,ANDROID_CONTROL,6314464,0.138007,nan%,1.808508,nan%,6.629633,nan%,251.196263,nan%
3,2021-05,ANDROID_SHOW,120668163,0.139229,0.89%***,1.796113,-0.69%***,6.592494,-0.56%***,249.931959,-0.5%***
5,2021-05,IOS_CONTROL,3521527,0.120842,nan%,1.656361,nan%,4.978854,nan%,185.987011,nan%
3,2021-05,IOS_SHOW,66655314,0.121703,0.71%,1.638625,-1.07%***,4.978942,0.0%,186.538293,0.3%***
7,2021-06,ANDROID_CONTROL,5366595,0.149538,nan%,1.925815,nan%,6.427937,nan%,312.218299,nan%
0,2021-06,ANDROID_SHOW,102923157,0.151884,1.57%***,1.917672,-0.42%,6.402566,-0.39%***,312.416176,0.06%


In [73]:
og[['month', 'bucket', 'dusers']+temp_cols].reset_index().style.applymap(color_pval)

,index,month,bucket,dusers,avg_orders,avg_orders_diff2,avg_gmv,avg_gmv_diff2,avg_pdp,avg_pdp_diff2,avg_imp,avg_imp_diff2
0,5,2021-04,ANDROID_CONTROL,6328013,0.151861,nan%,1.971166,nan%,7.006181,nan%,264.438004,nan%
1,4,2021-04,ANDROID_SHOW,120644425,0.152668,0.53%***,1.949224,-1.11%***,6.979089,-0.39%***,263.182445,-0.47%***
2,4,2021-04,IOS_CONTROL,3565405,0.126766,nan%,1.695718,nan%,5.285319,nan%,196.820919,nan%
3,0,2021-04,IOS_SHOW,67405117,0.127982,0.96%***,1.682146,-0.8%,5.271696,-0.26%,196.133093,-0.35%***
4,1,2021-05,ANDROID_CONTROL,6314464,0.138007,nan%,1.808508,nan%,6.629633,nan%,251.196263,nan%
5,3,2021-05,ANDROID_SHOW,120668163,0.139229,0.89%***,1.796113,-0.69%***,6.592494,-0.56%***,249.931959,-0.5%***
6,5,2021-05,IOS_CONTROL,3521527,0.120842,nan%,1.656361,nan%,4.978854,nan%,185.987011,nan%
7,3,2021-05,IOS_SHOW,66655314,0.121703,0.71%,1.638625,-1.07%***,4.978942,0.0%,186.538293,0.3%***
8,7,2021-06,ANDROID_CONTROL,5366595,0.149538,nan%,1.925815,nan%,6.427937,nan%,312.218299,nan%
9,0,2021-06,ANDROID_SHOW,102923157,0.151884,1.57%***,1.917672,-0.42%,6.402566,-0.39%***,312.416176,0.06%


# Why incremental during March, April, and May 2021?